In [ ]:
import pandas as pd 
import numpy as np
import glob  
import warnings

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

#################################################### CARERGANDO A BASE QUE FOI NORMALIZADA QUE COMPREENDE A TODAS AS EQUIPES DO HUGO (CLAROS/NET/GEVENUE)

caminho = r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES SEMANA PADRAO"
arquivos = glob.glob(caminho + "/*.csv")
lista = []
for arquivo in arquivos:
    print(arquivo)
    lista.append(arquivo)
    
    dfList = []
    
    for item in lista:   
        chunks = pd.read_csv(item, encoding='iso-8859-1', chunksize=1000000, sep=";", dtype='str')
        for df in chunks:
            dfList.append(df)

# Criando DataFrame com concat e a lista criada a cima
df_telecobs_hugo = pd.concat(dfList, sort=False, ignore_index=False)

In [ ]:
#### Filtrando somente coluna telefone dos contratantes para os telefones que estiverem na base do contratante seja removido da base de higienizados
remover_telefones_contratante_do_higienizado = df_telecobs_hugo.loc[:,['TELEFONE']].drop_duplicates(['TELEFONE'])
## Criando coluna para identificar na base higienizados os telefones que são iguais aos contratante
remover_telefones_contratante_do_higienizado = remover_telefones_contratante_do_higienizado.assign(BASE_TEL_CONTRATANTE="CONTRATANTE")

In [ ]:
#################################################### CARERGANDO A BASE DE TELEFONES HIGIENIZADOS

caminho = r"R:\TI\TELEFONIA\BASES_HIGIENIZADAS_CLARO_E_NET\TELEFONES CLARO E NET"
arquivos = glob.glob(caminho + "/*.csv")
lista = []
for arquivo in arquivos:
    print(arquivo)
    lista.append(arquivo)
    
    dfList = []
    
    for item in lista:   
        chunks = pd.read_csv(item, encoding='iso-8859-1', chunksize=1000000, sep=";", dtype='str')
        for df in chunks:
            dfList.append(df)

# Criando DataFrame com concat e a lista criada a cima
telefones_higienizados = pd.concat(dfList, sort=False, ignore_index=False)
telefones_higienizados = telefones_higienizados.rename(columns={'TELEFONE':'TELEFONE_HIGIENIZADO'})

telefones_higienizados = pd.merge(telefones_higienizados, remover_telefones_contratante_do_higienizado, right_on=['TELEFONE'], left_on=['TELEFONE_HIGIENIZADO'], how="left").query("BASE_TEL_CONTRATANTE != 'CONTRATANTE'")\
.loc[:,['CPF','NOME','TELEFONE_HIGIENIZADO']]


In [ ]:
######################################################  BASE NORMALIZADA X TELEFONES HIGIENIZADOS  ######################################################

df_telecobs_hugo = pd.merge(df_telecobs_hugo, telefones_higienizados, right_on=['CPF'], left_on=['CPF'], how="left")\
.loc[:,['CPF','DEVEDOR','NOME_x','TELEFONE','EMAIL','ATRASO','SALDO_ABERTO','BASE','CONTRATO','TELEFONE_HIGIENIZADO']]

### REMOVENDO DUPLICADOS DEPOIS DO CRUZAMENTO, PARA EVITAR NOS PROXIMOS CRUZAMENTOS TER VARIOS CPF'S IGUAIS DESNCESSARIAMENTE.
df_telecobs_hugo = df_telecobs_hugo.drop_duplicates(['CPF','DEVEDOR', 'NOME_x', 'TELEFONE', 'EMAIL','ATRASO', 'BASE', 'TELEFONE_HIGIENIZADO'])

In [ ]:
#################################################### CARERGANDO A BASE DE EMAILS HIGIENIZADOS

caminho = r"R:\TI\TELEFONIA\BASES_HIGIENIZADAS_CLARO_E_NET\EMAILS CLARO E NET"
arquivos = glob.glob(caminho + "/*.csv")
lista = []
for arquivo in arquivos:
    print(arquivo)
    lista.append(arquivo)
    
    dfList = []
    
    for item in lista:   
        chunks = pd.read_csv(item, encoding='iso-8859-1', chunksize=1000000, sep=";", dtype='str')
        for df in chunks:
            dfList.append(df)

# Criando DataFrame com concat e a lista criada a cima
emails_higienizados = pd.concat(dfList, sort=False, ignore_index=False)
emails_higienizados = emails_higienizados.rename(columns={'EMAIL':'EMAIL_HIGIENIZADO'})

In [ ]:
######################################################  BASE NORMALIZADA X EMAILS HIGIENIZADOS  ######################################################
df_telecobs_hugo = pd.merge(df_telecobs_hugo, emails_higienizados, right_on=['CPF'], left_on=['CPF'], how="left")\
.loc[:,['CPF','DEVEDOR','NOME_x','TELEFONE','EMAIL','ATRASO','SALDO_ABERTO','BASE','CONTRATO','TELEFONE_HIGIENIZADO','EMAIL_HIGIENIZADO']]

### REMOVENDO DUPLICADOS DEPOIS DO CRUZAMENTO, PARA EVITAR NOS PROXIMOS CRUZAMENTOS TER VARIOS CPF'S IGUAIS DESNCESSARIAMENTE.
df_telecobs_hugo = df_telecobs_hugo.drop_duplicates(['CPF','DEVEDOR', 'NOME_x', 'TELEFONE', 'EMAIL','ATRASO', 'BASE', 'TELEFONE_HIGIENIZADO', 'EMAIL_HIGIENIZADO'])

In [ ]:
#################################################### CARERGANDO A BASE DE VALIDADOS

caminho = r"R:\TI\TELEFONIA\BASE DE VALIDADOS"
arquivos = glob.glob(caminho + "/*.csv")
lista = []
for arquivo in arquivos:
    print(arquivo)
    lista.append(arquivo)
    
    dfList = []
    
    for item in lista:   
        chunks = pd.read_csv(item, encoding='iso-8859-1', chunksize=1000000, sep=";", dtype='str',usecols=['Numero', 'Usa whatsapp', 'data de envio'])
        for df in chunks:
            dfList.append(df)

# Criando DataFrame com concat e a lista criada a cima
telefones_validados = pd.concat(dfList, sort=False, ignore_index=False)

In [ ]:
######################################################  BASE NORMALIZADA X TELEFONES INTERSIC E GEVENUE  ######################################################

df_telecobs_hugo = pd.merge(df_telecobs_hugo, telefones_validados, right_on=['Numero'], left_on=['TELEFONE'], how="left")\
.loc[:,['CPF','DEVEDOR','NOME_x','TELEFONE','Usa whatsapp','data de envio','EMAIL','ATRASO','SALDO_ABERTO','BASE','CONTRATO','TELEFONE_HIGIENIZADO','EMAIL_HIGIENIZADO']]

############ APÓS O CRUZAMENTO, MARCAMOS COMO "NAO_VALIDADO" A BASE QUE NÃO FOI ENCONTRADA NO BANCO DE VALIDOS OU NAO.
########## RENOMEANDO A COLUNA SE USA WHATSS PARA IDENTIFICAR QUE A VALIDAÇÃO É REFERENTE A BASE DE CONTRATANTE / DATA DA VALIDACAO TAMBEM
df_telecobs_hugo['Usa whatsapp'] = df_telecobs_hugo['Usa whatsapp'].fillna('Nao_Validado')
df_telecobs_hugo = df_telecobs_hugo.rename(columns={'Usa whatsapp':'BASE_CONTRATANTE_VALIDADOS'})
df_telecobs_hugo = df_telecobs_hugo.rename(columns={'data de envio':'DATA_VALIDACAO_CONTRATANTE'})

######################################################  BASE NORMALIZADA X TELEFONES HIGIENIZADOS  ######################################################

df_telecobs_hugo = pd.merge(df_telecobs_hugo, telefones_validados, right_on=['Numero'], left_on=['TELEFONE_HIGIENIZADO'], how="left")\
.loc[:,['CPF','DEVEDOR','NOME_x','TELEFONE','BASE_CONTRATANTE_VALIDADOS','DATA_VALIDACAO_CONTRATANTE','EMAIL','ATRASO','SALDO_ABERTO','BASE','CONTRATO',
        'TELEFONE_HIGIENIZADO','Usa whatsapp','data de envio','EMAIL_HIGIENIZADO']]

############ APÓS O CRUZAMENTO, MARCAMOS COMO "NAO_VALIDADO" A BASE QUE NÃO FOI ENCONTRADA NO BANCO DE VALIDOS OU NAO.
df_telecobs_hugo['Usa whatsapp'] = df_telecobs_hugo['Usa whatsapp'].fillna('Nao_Validado')
############ APÓS O CRUZAMENTO, MARCAMOS COMO "NAO_VALIDADO" A BASE QUE NÃO FOI ENCONTRADA NO BANCO DE VALIDOS OU NAO.
########## RENOMEANDO A COLUNA SE USA WHATSS PARA IDENTIFICAR QUE A VALIDAÇÃO É REFERENTE A BASE DE CONTRATANTE / DATA DA VALIDACAO TAMBEM
df_telecobs_hugo = df_telecobs_hugo.rename(columns={'Usa whatsapp':'BASE_HIGI_VALIDADOS'})
df_telecobs_hugo = df_telecobs_hugo.rename(columns={'data de envio':'DATA_VALIDACAO_HIGI'})

In [ ]:
### REMOVENDO DUPLICADOS DEPOIS DO CRUZAMENTO, PARA EVITAR NOS PROXIMOS CRUZAMENTOS TER VARIOS CPF'S IGUAIS DESNCESSARIAMENTE.
df_telecobs_hugo = df_telecobs_hugo.drop_duplicates(['CPF','DEVEDOR', 'NOME_x', 'TELEFONE', 'EMAIL','ATRASO', 'BASE','CONTRATO', 'TELEFONE_HIGIENIZADO','EMAIL_HIGIENIZADO'])
df_telecobs_hugo

In [ ]:
df_telecobs_hugo.to_csv(r"R:\TI\TELEFONIA\BASES CLARO E NET ATIVA\BASES SEMANA PADRAO/BASE_OPERACIONAL_TELECOB_HUGO.txt",index=False, sep=';')